In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os

# GPU memory limit
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import lib.psnr_ssim as metrics
import lib.training_loops as tr


2021-10-15 02:38:50.771519: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-10-15 02:38:51.711236: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-15 02:38:51.711815: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-15 02:38:51.744580: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 02:38:51.744815: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 119.24GiB/s
2021-10-15 02:38:51.744828: I tensorflow/stream_executor/platform/def

1 Physical GPUs, 1 Logical GPUs


## Cargar modelo para su evaluación

In [2]:
# La función de tensorflow de cargar modelos a partir de archivos json requiere que, en caso de que una red contenga clases customizadas,
# se añada como parámetro un argumento que indique donde se encuentra la definición de cada una de esas clases
# Con el siguiente código, se creará un diccionario que contendrá todas las clases definidas en el módulo training_loops, permitiendo así
# cargar modelos que fueron creados empleando uno de estos métodos
training_loops = dir(tr)
training_loops = list(filter(lambda x: x != 'tf' and x != 'keras' and not x.startswith('__'), training_loops))

custom_objects = {training_loop: eval('tr.' + training_loop) for training_loop in training_loops}

def load_model(path_structure, path_weights):
   
    with open(path_structure, 'r') as json_file:
        model = tf.keras.models.model_from_json(json_file.read(), custom_objects=custom_objects)
    
    model.load_weights(path_weights)
    
    return model

In [3]:
saved_models_dir = 'TRAINED_MODELS_NEW/'

print('Model structures')
print(os.listdir(saved_models_dir))

Model structures
['RCAN-SCALE3-SHORT', 'RCAN-BASE-SCALE3', 'RCAN-SCALE3']


In [4]:
# Select structure
structure = 'RCAN-SCALE3'

# Select training
saved_models_of_structure_dir = saved_models_dir + structure + '/'
print('Trainings')
print([x for x in os.listdir(saved_models_of_structure_dir) if x.endswith('.json')])

Trainings
['model_replica_original.json', 'model_replica_original_continuacion.json', 'model_loss-computed-on-lr-and-hr.json', 'model_loss-computed-on-lr.json']


In [5]:
# Selected training
# In fact, any json file should work. They all should have the same content
model_structure = 'model_replica_original_continuacion'

# Select model (choose metric)
# Every training saves 2 models: best validation psnr and best validation ssim

best_metric = 'ssim'

model_weights = model_structure + '_best_' + best_metric + '.h5'

print('Selected weights configuration:', model_weights)

# Load model
model = load_model(saved_models_of_structure_dir + model_structure + '.json', saved_models_of_structure_dir + model_weights)

Selected weights configuration: model_replica_original_continuacion_best_ssim.h5


In [6]:
# Auxiliar functions
def make_divisible_by_scale(img, scale):
    return img[:img.shape[0]-img.shape[0]%scale, :img.shape[1]-img.shape[1]%scale, :]

read_img = lambda path: cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

write_img = lambda path, img: cv2.imwrite(path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

In [7]:
# Evaluate on test datasets
TEST_DATASETS = ['Set5', 'Set14', 'Urban100', 'Manga109']

SCALE = 3

save_superresolved_images = False

if save_superresolved_images and not os.path.exists('superresolved_datasets/'):
    os.mkdir('superresolved_datasets/')

for dataset in TEST_DATASETS:

    hr_path = 'data/' + dataset + '/'
    lr_path = 'data/lr_images/' + dataset + '/'
    
    if save_superresolved_images and not os.path.exists('superresolved_datasets/' + dataset + '_sr/'):
        os.mkdir(dataset + '_sr')
    
    results = {}
    psnr_list = []
    ssim_list = []
    
    for img in os.listdir(hr_path):
        
        hr = read_img(hr_path + img)
        hr = make_divisible_by_scale(hr, SCALE)
        
        lr = read_img(lr_path + img)
        
        # THIS ONLY WORKS IF MODEL INPUT HAS 3 CHANNELS
        sr = model(np.expand_dims(lr, 0), training=False)[0,...].numpy()
        
        sr = np.round(np.clip(sr, 0, 255)).astype(np.uint8)
        
        if save_superresolved_images:
            write_img('superresolved_datasets/' + dataset + '_sr/' + img, sr)

        psnr, ssim = metrics.compare_measure(hr, sr)
        
        img_name = img.split('.')[0]
        
        results[img_name] = {'psnr': psnr, 'ssim': ssim}
        
        psnr_list.append(psnr)
        ssim_list.append(ssim)
    
    # Average results
    print(dataset)
    print('psnr:', sum(psnr_list) / len(psnr_list), 'ssim:', sum(ssim_list) / len(ssim_list))

2021-10-15 02:39:00.548241: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-15 02:39:00.568482: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599990000 Hz
2021-10-15 02:39:00.575914: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-10-15 02:39:01.469755: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-10-15 02:39:01.503261: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-10-15 02:39:01.829169: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


/media/miguel/Nuevo SSD/Super-Resolution-copiaRemoto/lib/psnr_ssim.py:144: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = ar[slices]


2021-10-15 02:39:22.072125: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


2021-10-15 02:39:24.353385: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.56GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-10-15 02:39:24.807118: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.37GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


Set5
psnr: 34.68228624710152 ssim: 0.929971858349696
